In [1]:
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')

In [2]:
# 现在可以安全地使用环境变量了
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from PK_LLM_endfront.QA_generation.chains import make_Q_chain,make_RAG_chain,check_QA

![](P\whiteboard_exported_image.png)

In [3]:
def generate_Q(t:str,a:str)->list:
    QA_Gen_Chain = make_Q_chain()
    r= QA_Gen_Chain.invoke({
        "title":t,
        "abstract":a
    })
    return r.Question
def generate_A(pdf_path:str,q_lis:list[str])->dict:
    rag_chain = make_RAG_chain(pdf_path) # 有成本
    dir = []
    for _q in q_lis:
        answer = rag_chain.invoke(_q)
        #print("问题为:\n",_q,"\n\n ")
        #print("\n答案为:\n",answer.Answer,"\n\n ")
        #print("\n答案来源为:\n",answer.Source_context,"\n\n ")
        #print("---------------------------")
        dir.append({"Question":_q,"Answer":answer.Answer,"Source_context":answer.Source_context})
    return dir

In [4]:
# 测试demo
"""
import pandas as pd
path = "_data/1_Analytical_Methods"
data = pd.read_csv(f"{path}/litera_lis.csv",index_col=0)
data.head(2)
r = generate_Q(data.loc['SL01-2-J']['Title'],data.loc['SL01-2-J']['Abstract'])
a = generate_A(f'{path}/literature_PDF/SL01-2-J.pdf',r)
from pprint import pprint
pprint(a)
"""

'\nimport pandas as pd\npath = "_data/1_Analytical_Methods"\ndata = pd.read_csv(f"{path}/litera_lis.csv",index_col=0)\ndata.head(2)\nr = generate_Q(data.loc[\'SL01-2-J\'][\'Title\'],data.loc[\'SL01-2-J\'][\'Abstract\'])\na = generate_A(f\'{path}/literature_PDF/SL01-2-J.pdf\',r)\nfrom pprint import pprint\npprint(a)\n'

In [5]:
import json

class qa():
    def __init__(self, q:str, a:str, s:str, doi:str,_id:str,time:str) -> None:
        self.Question = q
        self.Answer = a
        self.Source_context = s
        self.DOI = doi
        self._id = _id
        self.time = time
    def add_score(self, score:int):
        self.score = score
    def to_dict(self):
        return {
            'Question': self.Question,
            'Answer': self.Answer,
            'Source_context': self.Source_context,
            'DOI': self.DOI,
            'time' :str(self.time),
            'Score': getattr(self, 'score', None)  # 如果分数不存在，则返回None
        }
    @staticmethod
    def from_dict(data):
        qa_instance = qa(data['Question'], data['Answer'], data['Source_context'], data['DOI'],data['time'])
        if 'Score' in data:
            qa_instance.add_score(data['Score'])
        return qa_instance

class recorder():
    def __init__(self) -> None:
        self.qa = []
        self.qa_tem = []
        self.good_qa = []
    
    def create_qa_temp(self, q, a, s, doi,_id,time):
        QA = qa(q, a, s, doi,_id,time)
        self.qa_tem.append(QA)
    
    def score_temp_qa(self, score_lis:list[int]):
        max_s = 0
        max_i = 0
        for i, _s in enumerate(score_lis):
            self.qa_tem[i].add_score(_s)
            if _s > max_s:
                max_s = _s
                max_i = i
        self.qa += self.qa_tem
        self.good_qa.append(self.qa_tem[max_i])
        self.qa_tem = []

    def save_json(self, qa_path="qa_data.json", good_qa_path='good_qa_data.json'):
        with open(qa_path, 'w', encoding='utf-8') as file:
            json.dump([qa_obj.to_dict() for qa_obj in self.qa], file, ensure_ascii=False, indent=4)
        with open(good_qa_path, 'w', encoding='utf-8') as file:
            json.dump([qa_obj.to_dict() for qa_obj in self.good_qa], file, ensure_ascii=False, indent=4)

    def load_json(self, qa_path="qa_data.json", good_qa_path='good_qa_data.json'):
        with open(qa_path, 'r', encoding='utf-8') as file:
            self.qa = [qa.from_dict(data) for data in json.load(file)]
        with open(good_qa_path, 'r', encoding='utf-8') as file:
            self.good_qa = [qa.from_dict(data) for data in json.load(file)]

# 使用示例
# rec = recorder()
# rec.load_json('qa_data.json', 'good_qa_data.json')


In [6]:
import threading
import sys

def timeout(seconds, error_message='Function call timed out'):
    def decorator(func):
        def _handle_timeout():
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            timer = threading.Timer(seconds, _handle_timeout)
            timer.start()
            try:
                result = func(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return wrapper
    return decorator


In [7]:
import pandas as pd
from tqdm import tqdm
import warnings
# 忽略所有警告
warnings.simplefilter('ignore')

import pandas as pd

In [8]:
# 应用超时装饰器
@timeout(300)  
def process_index(df:pd.DataFrame,_index,_recorder:recorder,_id,_pdf_path):
    try:
        _recorder.load_json(qa_path=f"_data/QA/qa_data{_id}.json", good_qa_path=f'_data/QA/good_qa_data{_id}.json')
    except:
        pass
    if df.loc[_index]['is_download'] !=1:
        return 
    t = df.loc[_index]['Title']
    a = df.loc[_index]['Abstract']
    doi =  df.loc[_index]['DOI']
    time = df.loc[_index]['Publication Year']
    q_lis  = generate_Q(t,a)
    pdf_path = f"{_pdf_path}/{_index}.pdf"
    a_dir = generate_A(pdf_path,q_lis)
    for _ in a_dir:
        _recorder.create_qa_temp(q=_["Question"],
                                a=_['Answer'],
                                s=_['Source_context'],
                                doi=doi,
                                _id=_index,
                                time=time)
    from langchain_openai import ChatOpenAI
    r =check_QA(a_dir,llm =ChatOpenAI(model='gpt-4',temperature=0.5))
    # print(r)
    _recorder.score_temp_qa(r.result)
    _recorder.save_json(qa_path=f"_data/QA/qa_data{_id}.json", good_qa_path=f'_data/QA/good_qa_data{_id}.json')


def QA_G(df:pd.DataFrame,index_parts,_pdf_path,_id = 0):
    _recorder = recorder()
    for _index in index_parts:
        if df.loc[_index].get("Abstract") == "No abstract available for this PMID.":
            continue
        try:
            process_index(df,_index,_recorder,_id,_pdf_path)
        except Exception as e:
            print(f"Error processing index {_index}: {e}")
            continue

In [9]:
import numpy as np
def main_run(_name,num_index:int,_i:str ="01"):
    path = f"_data/{_name}"
    _pdf_path = f"{path}/literature_PDF"
    df = pd.read_csv(f"{path}\litera_lis.csv",index_col=0)
    df = df[df['is_download']==1]
    index_parts = np.array_split(df.index, 30)
    # print(df.index[155:])
    QA_G(df,index_parts[num_index],_pdf_path = _pdf_path,_id=_i)

In [10]:
from tqdm import tqdm
name = ["1_Analytical_Methods",
        "2_Environmenta_Exposure",
        "3_Environmental_Behavior",
        "4_Biological_Behavior",
        "5_Toxicity",
        "6_Health Risk"]

for i in tqdm(range(30)):
    print(i)
    if i < 0:
        continue
    if i <1:
        _id = "0"+str(i)
    else:
        _id = str(i)
    # main_run(name[5],num_index=i,_i=_id)

  0%|          | 0/30 [00:00<?, ?it/s]

0


  3%|▎         | 1/30 [02:34<1:14:53, 154.95s/it]

1


  7%|▋         | 2/30 [04:11<56:17, 120.62s/it]  

...

 93%|█████████▎| 28/30 [1:03:48<03:48, 114.16s/it]

28


 97%|█████████▋| 29/30 [1:05:47<01:55, 115.43s/it]

29


100%|██████████| 30/30 [1:08:05<00:00, 136.19s/it]


生成结束，开始整理并且可视化